<a href="https://colab.research.google.com/github/FREDY129053/SkillBox_ML/blob/main/8_8_ML_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практическая работа

## Цель практической работы

Вы уже делали задание, в котором с помощью метода ближайших соседей оценивали склонность клиента банка откликнуться или не откликнуться на предложение.

В этом практическом задании ваши цели:
*  решить эту же задачу с помощью логистической регрессии;
*  потренироваться в подборе порога;
*  потренироваться в подборе гиперпараметров модели.

## Что входит в работу

*  Загрузить данные для задачи.
*  Обучить метод ближайших соседей с заданным количеством соседей k, вычислить метрики.
*  Обучить логистическую регрессию с параметрами по умолчанию, вычислить метрики.
*  Подобрать порог модели, вычислить метрики.
*  Подобрать гиперпараметр С (константа регуляризации) модели, вычислить метрики.

## Что оценивается

*  Выполнены все этапы задания: код запускается, отрабатывает без ошибок; подробно и обоснованно написаны текстовые выводы, где это требуется.

## Формат сдачи
Выполните предложенные задания — впишите свой код (или, если требуется, текст) в ячейки после комментариев.

*Комментарии — это текст, который начинается с символа #. Например: # ваш код здесь.*

Сохраните изменения, используя опцию Save and Checkpoint из вкладки меню File или кнопку Save and Checkpoint на панели инструментов. Итоговый файл в формате .ipynb (файл Jupyter Notebook) загрузите в личный кабинет и отправьте на проверку.

In [1]:
# подключим библиотеки
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('/content/ClientsData.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


В этом ноутбуке нам придётся подбирать гиперпараметры модели, а ещё порог. Поэтому, чтобы не переобучиться, разобъём данные на трейн, валидацию и тест.

*  Обучать модели будем на тренировочных данных.
*  Подбирать необходимые величины — по валидации.
*  Оценивать качество — на тесте.

In [4]:
# разделим данные на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

В задании по методу ближайших соседей было найдено, что оптимальное число соседей k = 9.

Обучите на тренировочных данных KNN с k = 9 и выведите матрицу ошибок, а также значение метрик precision и recall на тестовых данных.

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score

In [7]:
# ваш код здесь
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(Xtrain, ytrain)

predictions = knn.predict(Xtest)

print('Матрица ошибок')
print(confusion_matrix(ytest, predictions))

print('\n\nPrecision', precision_score(ytest, predictions))

print('\n\nRecall', recall_score(ytest, predictions))

Матрица ошибок
[[3981   27]
 [ 555    4]]


Precision 0.12903225806451613


Recall 0.007155635062611807


Какой вывод можно сделать:
- для класса 0 — клиент не откликнулся — мы получили достаточно высокие значения TP в том числе потому, что представителей этого класса больше;
- для класса 1 — клиент откликнулся — мы получили низкие значения TN.

Поэтому значения precision и recall низкие. Модель даёт неудовлетворительные результаты, так как находит мало клиентов, которые откликнутся на предложение.



Обучите логистическую регрессию с параметрами по умолчанию и посмотрите на метрики.

Везде дальше при оценке метрик надо выводить confusion_matrix, precision и recall.

In [12]:
def print_metrics(test, pred):
  print('Матрица ошибок')
  print(confusion_matrix(test, pred))

  print('\n\nPrecision', precision_score(test, pred))

  print('\n\nRecall', recall_score(test, pred))

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# ваш код здесь
logreg = LogisticRegression()
logreg.fit(Xtrain, ytrain)

predictions = logreg.predict(Xtest)

print_metrics(ytest, predictions)
print('\n\nAccuracy: ', accuracy_score(ytest, predictions))

Матрица ошибок
[[4008    0]
 [ 559    0]]


Precision 0.0


Recall 0.0


Accuracy:  0.8776001751696957


Наша цель — найти как можно больше клиентов, которые откликнутся на предложение. А модель таких не находит.

Мы помним, что метод predict_proba у логистической регрессии предсказывает математические (то есть корректные) вероятности классов. Предскажите вероятности классов с помощью обученной логистической регрессии на тестовых данных и выведите вероятности положительного класса для первых десяти объектов.

Глядя на полученные вероятности, попробуйте объяснить, почему вы получили именно такую матрицу ошибок и такие значения точности с полноты.

In [25]:
# ваш код здесь
probs = logreg.predict_proba(Xtest)[:, 1]
probs[:10].round(5)

array([0.14042, 0.14107, 0.19378, 0.27152, 0.12612, 0.04103, 0.04262,
       0.03656, 0.0823 , 0.06786])

In [ ]:
# ваше объяснение здесь
# Маленькие вероятности отрицательного класса, поэтому ни один класс отрицательный не был угадан

Давайте уточним цель. Пусть нам нужно найти как можно больше клиентов, которые откликнутся на предложение, то есть максимизировать полноту (recall).

При этом хочется, чтобы точность модели (precision) не была очень низкой. Заказчик посмотрел на результаты работы KNN и потребовал, чтобы точность была не ниже, чем у KNN: $precision \geq 0.13$.

Давайте будем изменять порог для перевода вероятности в классы так, чтобы:
*   максимизировать значение recall
*   при условии, что $precision \geq 0.13$.

Если мы будем подбирать порог по тестовой выборке, то, по сути, обучимся на ней и, значит, переобучимся. Это плохо.

Поэтому предскажите вероятности на валидационной выборке и подберите порог по ней (Xval, yval), а затем посмотрите, какое качество для найденного порога вы получите на тестовых данных.

In [26]:
probs_val = logreg.predict_proba(Xval)[:, 1] # ваш код здесь

max_recall = -1
thr = -1
prec = -1

for threshold in np.arange(0.05, 1, 0.001):
    # для каждого значения порога переведите вероятности в классы
    # посчитайте метрики
    yval_pred = (probs_val >= threshold).astype(int)
    precision = precision_score(yval, yval_pred)
    recall = recall_score(yval, yval_pred)
    if recall > max_recall and precision >= 0.13:
      max_recall = recall
      thr = threshold
      prec = precision

# напечатайте порог, для которого получается максимальная полнота, при precision >= 0.13
print(f'Порог: {thr}\n')
print(f'Полнота: {recall}\n')
print(f'Точность: {prec}\n')

Порог: 0.07300000000000002

Полнота: 0.0

Точность: 0.1310285958173282



In [27]:
# по найденному порогу переведите вероятности в классы на тесте и напечатайте метрики
probs_test = logreg.predict_proba(Xtest)[:, 1]
ytest_pred = (probs_test >= thr).astype(int)

print_metrics(ytest, ytest_pred)

Матрица ошибок
[[1152 2856]
 [ 104  455]]


Precision 0.13742071881606766


Recall 0.813953488372093


Сделайте вывод. Смогли ли мы с помощью подбора порога добиться большего значения recall, чем у KNN?

In [ ]:
# ваш вывод здесь
# Да, смог. Нахождение параметров позволило увеличить полноту при точности >= 0.13

А ещё, чтобы улучшить качество предсказания, можно подбирать значение гиперпараметра C у логистической регрессии. Для каждого значения C придётся подбирать свой порог, поэтому:  

1. Обучите для значений C из диапазона [0.05, 0.15, 0.25, ...., 10.05] логистическую регрессию (на тренировочных данных).

2. Для каждой из обученных моделей во внутреннем цикле подберите оптимальный порог (как в предыдущем задании) — на валидационных данных.



В качестве результата выведите значение C и порога для модели, которая даёт наилучшие значения метрик (наибольший recall при ограничении на $precision \geq 0.13$).

Также напечатайте полученные метрики (матрицу ошибок, точность и полноту) для лучшей модели — на тестовых данных.

In [28]:
regs = []
recalls = []
thresholds = []
precisions = []

for reg in np.arange(0.001, 1, 0.01):

    # обучите логистическую регрессию с C=reg
    logreg = LogisticRegression(C=reg)
    logreg.fit(Xtrain, ytrain)
    probs_val = logreg.predict_proba(Xval)[:, 1]

    max_recall = -1
    thr = -1
    prec = -1

    for threshold in np.arange(0.05, 0.25, 0.001):
        # подберите оптимальный порог как в задании выше
        yval_pred = (probs_val >= threshold).astype(int)
        precision = precision_score(yval, yval_pred)
        recall = recall_score(yval, yval_pred)
        if recall > max_recall and precision >= 0.13:
          max_recall = recall
          thr = threshold
          prec = precision

    recalls.append(max_recall)
    thresholds.append(thr)
    precisions.append(prec)
    regs.append(reg)

In [29]:
# выведите значения C, precision, recall, threshold для наилучшей по заданным условиям модели
id = np.argmax(recalls)

print('Best C = ', regs[id])
print('Best precision = ', precisions[id])
print('Best recall = ', recalls[id])
print('Best thresholds = ', thresholds[id])

Best C =  0.001
Best precision =  0.1305827307528711
Best recall =  0.8696883852691218
Best thresholds =  0.07300000000000002


In [30]:
# с помощью найденных C и threshold обучите модель на тренировочных данных, сделайте предсказание на тесте и по найденному порогу получите классы
# напечатайте метрики
logreg = LogisticRegression(C=regs[id])
logreg.fit(Xtrain, ytrain)
probs_test = logreg.predict_proba(Xtest)[:, 1]
ytest_pred = (probs_test >= thresholds[id]).astype(int)

print_metrics(ytest, ytest_pred)

Матрица ошибок
[[1143 2865]
 [ 102  457]]


Precision 0.13756773028296207


Recall 0.817531305903399


Влияет ли изменение гиперпараметра C на качество модели (и, соответственно, метрики) в этой задаче?

In [ ]:
# ваш вывод здесь
# Да. Добился лучшей полноты при заданном уровне точности

Ответьте развёрнуто на следующие вопросы:

* Удалось ли при помощи логистической регрессии и подбора порога превзойти качество метода ближайших соседей в этой задаче?

* Смогли ли мы при помощи этой модели получить высокий recall при ограничениях, поставленных заказчиком?


In [ ]:
# ваш вывод здесь
# Вроде смог. Улучшил полноту это точно
# Да, смог